In [1]:
%reload_ext autoreload
%autoreload 2
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from utilities import data_handler_old as data_handler
import numpy as np
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer # for tokenization only



from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# THE CELL BELLOW CONTAINS THE FILENAMES TO CHANGE

In [2]:
UNLABLED_DATA = '/Users/ianmagnusson/IITUDND/data/retrieved_data/tweets/harvey_extras.json'
LABLED_DATA = '/Users/ianmagnusson/IITUDND/data/CrisisMMD_v1.0/json/hurricane_harvey_final_data.json'
CLASS_DATA = '/Users/ianmagnusson/IITUDND/data/CrisisMMD_v1.0/annotations/hurricane_harvey_final_data.tsv'
NPY_OUTPUT_DIR = '/Users/ianmagnusson/IITUDND/data/extracted_features/combined_NLP/harvey/'

# get tweets from file and split into test/train

In [3]:
data = data_handler.DataHandler(UNLABLED_DATA,LABLED_DATA,CLASS_DATA)

train_labeled, train_histories, train_histories_by_target, test_labeled, test_histories, test_histories_by_target, train_merged, train_classes, test_classes = data.get_train_test_split()



## Set up glove

In [4]:
conversion_file = '../models/gensim_glove.txt'
# convert glove format to work with gensim. tutorial here https://radimrehurek.com/gensim/scripts/glove2word2vec.html
# _ = glove2word2vec('/Users/ianmagnusson/IITUDND/models/glove.twitter.27B.200d.txt', conversion_file)

In [5]:
# load model, NOTE this is very slow!
glove = KeyedVectors.load_word2vec_format(conversion_file)

# Extract features

# Class labels



In [24]:
# build class labels

y_train = np.array(train_classes)
y_test = np.array(test_classes)

In [25]:
# checkpoint!

np.save(NPY_OUTPUT_DIR + 'y_train.npy', y_train)
np.save(NPY_OUTPUT_DIR + 'y_test.npy', y_test)


#y_train = np.load(NPY_OUTPUT_DIR + 'y_train.npy')
#y_test = np.load(NPY_OUTPUT_DIR + 'y_test.npy')

# GLOVE

In [6]:
EMBED_DIM = 200

def embed_tweets(tweet_jsons):
    X_embedded = np.zeros((len(tweet_jsons),EMBED_DIM))
    tokenizer = CountVectorizer().build_tokenizer()
    for i, tweet_json in enumerate(tweet_jsons):
        text = tweet_json['text'].lower()
        tokens = [token for token in tokenizer(text) if token not in ENGLISH_STOP_WORDS]
        num_in_vocab = 0
        for token in tokens:
            if token in glove:
                X_embedded[i] += glove[token]
                num_in_vocab += 1
        X_embedded[i] = X_embedded[i] / num_in_vocab
    return X_embedded

def embed_histories(histories):
    X_embedded = np.zeros((len(histories),EMBED_DIM))
    tokenizer = CountVectorizer().build_tokenizer()
    for i, history in enumerate(histories):
        text = ' '.join([tweet_json['text'].lower() for tweet_json in history])
        tokens = [token for token in tokenizer(text) if token not in ENGLISH_STOP_WORDS]
        num_in_vocab = 0
        for token in tokens:
            if token in glove:
                X_embedded[i] += glove[token]
                num_in_vocab += 1
        X_embedded[i] = X_embedded[i] / num_in_vocab
    return X_embedded

In [7]:
# baseline features

X_labeled_train = embed_tweets(train_labeled)
X_histories_train = embed_histories(train_histories)
X_labeled_test = embed_tweets(test_labeled)
X_histories_test = embed_histories(test_histories)

In [8]:
# checkpoint!
np.save(NPY_OUTPUT_DIR + 'X_labeled_train.npy', X_labeled_train)
np.save(NPY_OUTPUT_DIR + 'X_histories_train.npy', X_histories_train)
np.save(NPY_OUTPUT_DIR + 'X_labeled_test.npy', X_labeled_test)
np.save(NPY_OUTPUT_DIR + 'X_histories_test.npy', X_histories_test)



#X_labeled_train = np.load(NPY_OUTPUT_DIR + 'X_labeled_train.npy')
#X_histories_train = np.load(NPY_OUTPUT_DIR + 'X_histories_train.npy')
#X_labeled_test = np.load(NPY_OUTPUT_DIR + 'X_labeled_test.npy')
#X_histories_test = np.load(NPY_OUTPUT_DIR + 'X_histories_test.npy')


In [9]:
# sequence features

def proccess_seq(histories_by_target):
    X_seq = [] # a list of 2d tensors of shape (len(seq), embed_dim)
    for history in histories_by_target:
        X_seq.append(embed_tweets(history))
    
    return X_seq

X_seq_train = proccess_seq(train_histories_by_target)
X_seq_test = proccess_seq(test_histories_by_target)

In [10]:
# checkpoint!
np.savez(NPY_OUTPUT_DIR + 'X_seq_glove_train.npz', *X_seq_train)
np.savez(NPY_OUTPUT_DIR + 'X_seq_glove_test.npz', *X_seq_test)



#X_seq_glove_train = np.load(NPY_INPUT_DIR + 'X_seq_glove_train.npz')
#X_seq_glove_test = np.load(NPY_INPUT_DIR + 'X_seq_glove_test.npz')

# TF-IDF

In [19]:
SVD_COMPONENTS = 200

def construct_vectorizer_and_SVD(merged):
    allTweets = []
    for i, tweet_json in enumerate(merged):
        text = tweet_json['text'].lower()
        allTweets.append(text)
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_merged = vectorizer.fit_transform(allTweets)
    svd = TruncatedSVD(n_components=SVD_COMPONENTS, n_iter=7, random_state=42)
    svd.fit(tfidf_merged)
    return vectorizer, svd


def vectorize_histories(histories, vectorizer, svd):
    rawHistories = [] # Will be in order
    for i, history in enumerate(histories):
        text = ' '.join([tweet_json['text'].lower() for tweet_json in history])
        rawHistories.append(text)
    histArr = vectorizer.transform(rawHistories)
    histFeatureArr = svd.transform(histArr)
    return histFeatureArr


def vectorize_tweets(tweets, vectorizer, svd):
    labeledTweets = []  # Will be in order
    for i, tweet_json in enumerate(tweets):
        text = tweet_json['text'].lower()
        labeledTweets.append(text)
    tweetArr = vectorizer.transform(labeledTweets)
    tweetFeatureArr = svd.transform(tweetArr)
    return tweetFeatureArr


In [20]:
# BASELINE

vectorizer, svd = construct_vectorizer_and_SVD(train_merged)
trainHistories = vectorize_histories(train_histories, vectorizer, svd)
trainTweets = vectorize_tweets(train_labeled, vectorizer, svd)
testHistories = vectorize_histories(test_histories, vectorizer, svd)
testTweets = vectorize_tweets(test_labeled, vectorizer, svd)


In [25]:
# Validation checks
print("Shape of train history np array:", trainHistories.shape)
print("Shape of train tweets np array):", trainTweets.shape)
print("Shape of test history np array:", testHistories.shape)
print("Shape of test tweets np array:", testTweets.shape)

Shape of train history np array: (3200, 200)
Shape of train tweets np array): (3200, 200)
Shape of test history np array: (800, 200)
Shape of test tweets np array: (800, 200)


In [22]:
# Save to outfiles
np.save(NPY_OUTPUT_DIR + 'trainHistories.npy', trainHistories)
np.save(NPY_OUTPUT_DIR + 'trainTweets.npy', trainTweets)
np.save(NPY_OUTPUT_DIR + 'testHistories.npy', testHistories)
np.save(NPY_OUTPUT_DIR + 'testTweets.npy', testTweets)
np.save(NPY_OUTPUT_DIR + 'trainClassifications.npy', train_classes)
np.save(NPY_OUTPUT_DIR + 'testClassifications.npy', test_classes)

In [23]:
# sequence features

def proccess_seq_tfidf(histories_by_target, vectorizer, svd):
    X_seq = [] # a list of 2d tensors of shape (len(seq), SVD_COMPONENTS)
    for history in histories_by_target:
        X_seq.append(vectorize_tweets(history, vectorizer, svd))
    return X_seq

X_seq_tfidf_train = proccess_seq_tfidf(train_histories_by_target, vectorizer, svd)
X_seq_tfidf_test = proccess_seq_tfidf(test_histories_by_target, vectorizer, svd)

In [28]:
# checkpoint!
np.savez(NPY_OUTPUT_DIR + 'X_seq_tfidf_train.npz', *X_seq_tfidf_train)
np.savez(NPY_OUTPUT_DIR + 'X_seq_tfidf_test.npz', *X_seq_tfidf_test)



#X_seq_tfidf_train = np.load(NPY_OUTPUT_DIR + 'X_seq_tfidf_train.npz')
#X_seq_tfidf_test = np.load(NPY_OUTPUT_DIR + 'X_seq_tfidf_test.npz')




